In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

api_keys={
    'google_api_key':os.getenv('google_api_key'),
    'pse':os.getenv('pse'),
    'openai_api_key':os.getenv('openai_api_key'),
    'composio_key':os.getenv('composio_api_key'),
    'notion_api_key':os.getenv('notion_integration_token')  # Add your Notion API key here
}

In [4]:
from composio_langgraph import Action, ComposioToolSet, App
from utils.composio_tools_agent import Composio_agent
from pydantic_ai import Agent, format_as_xml,RunContext
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider
from langchain_openai import ChatOpenAI
from datetime import datetime
from pydantic_ai.messages import ModelMessage
from pydantic import BaseModel,Field
from dataclasses import dataclass
import nest_asyncio
nest_asyncio.apply()

In [5]:
from pydantic_ai.mcp import MCPServerStreamableHTTP, MCPServerSSE, MCPServerStdio
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai import Agent
from pydantic_ai.messages import ModelMessage
from dataclasses import dataclass

@dataclass
class Message_state:
    messages:list[ModelMessage]

class NotionAgent_MCP:
    def __init__(self, mcp_server_url:str, api_keys:dict):
        self.mcp_server_url=mcp_server_url
        self.api_keys=api_keys
        self.tools=ComposioToolSet(api_key=api_keys['composio_key'])
        self.tool_shemas={
            'Notion Manager':{tool.name:tool for tool in self.tools.get_action_schemas(apps=[App.NOTION])}}
        self.memory=Message_state(messages=[])
        self.llm=OpenAIModel('gpt-4.1-nano',provider=OpenAIProvider(api_key=api_keys['openai_api_key']))
        self.mcp_server=MCPServerStreamableHTTP(self.mcp_server_url)
        self.agent=Agent(self.llm, mcp_servers=[self.mcp_server])

    async def main(self,query:str):
        async with self.agent.run_mcp_servers():  
            result = self.agent.run_sync(query, message_history=self.memory.messages)
            self.memory.messages=result.all_messages()
        return result.output

    def reset(self):
        self.memory.messages=[]

In [6]:
# agent2=NotionAgent_MCP(mcp_server_url=os.getenv('mcp_server_url'), api_keys=api_keys)

In [7]:
# await agent2.main('update it to add contents: "this is a test"')

In [8]:
from notion_agent import Notionagent


In [9]:
agent=Notionagent(api_keys=api_keys)

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Notion-agent\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions

Give Feedback / Get Help:
    On GitHub: https://github.com/ComposioHQ/composio/issues/new
    On Discord: https://dub.composio.dev/discord
    On Email: tech@composio.dev
    Talk to us on Intercom: https://composio.dev
    Book a call with us: https://composio.dev/redirect?url=https://calendly.com/composiohq/support?utm_source=py-sdk-logs&utm_campaign=calendly
If you need to debug this error, set `COMPOSIO_LOGGING_LEVEL=debug`.
  warnings.warn(
c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Notion-agent\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use

TypeError: [StructuredTool(name='NOTION_ADD_PAGE_CONTENT', description='Appends a single content block to a notion page or a parent block (must be page, toggle, to-do, bulleted/numbered list, callout, or quote); invoke repeatedly to add multiple blocks.', args_schema=<class 'composio.utils.shared.AddPageContentRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x000002443795E980>), StructuredTool(name='NOTION_CREATE_COMMENT', description='Adds a comment to a notion page (via `parent page id`) or to an existing discussion thread (via `discussion id`); cannot create new discussion threads on specific blocks (inline comments).', args_schema=<class 'composio.utils.shared.CreateCommentRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x000002443795E200>), StructuredTool(name='NOTION_CREATE_DATABASE', description='Creates a new notion 

In [12]:
agent.chat('hello')

  + Exception Group Traceback (most recent call last):
  |   File "c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Notion-agent\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "C:\Users\trist\AppData\Local\Temp\ipykernel_18904\1283318175.py", line 1, in <module>
  |     await agent.chat('hello')
  |   File "c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Notion-agent\notion_agent.py", line 46, in chat
  |     async with self.agent.run_mcp_servers():
  |                ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  |   File "C:\Users\trist\AppData\Local\Programs\Python\Python313\Lib\contextlib.py", line 214, in __aenter__
  |     return await anext(self.gen)
  |            ^^^^^^^^^^^^^^^^^^^^^
  |   File "c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Notion-agent\.venv\Lib\site-packages\pydantic_ai\agent.py", line 1692, in run_mcp_servers
  |     await exit_stack.enter_as